# WEATHER data from different stations
### Stations localized in: Warszawa, Wrocław, Szczecin, Rzeszów

In [1]:
import pandas as pd
from matplotlib import pyplot
import seaborn as sn
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import datetime as dt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import xgboost as xgb

# Read the CSV file
all_data = pd.read_csv("data_for_main_model/combined_cities_data_for_xgb.csv", delimiter=';')
all_data.head()

C:\Users\AWeg\AppData\Local\Temp\ipykernel_14852\2210513273.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv("data_for_main_model/combined_cities_data_for_xgb.csv", delimiter=';')


,station,valid,lon,lat,tmpc,relh,sped,day,month,year,hour,minutes
0,EPRZ,01.01.2015 00:30,22.0200,50.1114,-4.0,79.62,13.80,1,1,2015,0,30
1,EPWR,01.01.2015 00:30,16.8858,51.1027,2.0,86.59,6.90,1,1,2015,0,30
2,EPWR,01.01.2015 01:00,16.8858,51.1027,2.0,86.59,6.90,1,1,2015,1,0
3,EPRZ,01.01.2015 01:00,22.0200,50.1114,-4.0,79.62,12.65,1,1,2015,1,0
4,EPWA,01.01.2015 01:00,20.9611,52.1628,1.0,93.03,6.90,1,1,2015,1,0


### remove unnecessary columns

In [2]:
#remove unnecesseary columns
filtered = all_data[['station','lon', 'lat', 'tmpc', 'relh', 'sped', 'day', 'month', 'year', 'hour', 'minutes']]
filtered = filtered[2:].reset_index(drop=True)
filtered.head(20)

,station,lon,lat,tmpc,relh,sped,day,month,year,hour,minutes
0,EPWR,16.8858,51.1027,2.0,86.59,6.90,1,1,2015,1,0
1,EPRZ,22.0200,50.1114,-4.0,79.62,12.65,1,1,2015,1,0
2,EPWA,20.9611,52.1628,1.0,93.03,6.90,1,1,2015,1,0
3,EPSC,14.6228,53.3953,4.0,100.0,9.20,1,1,2015,1,0
4,EPSC,14.6228,53.3953,4.0,93.19,9.20,1,1,2015,1,30
5,EPRZ,22.0200,50.1114,-4.0,79.62,12.65,1,1,2015,1,30
6,EPWA,20.9611,52.1628,1.0,100.0,6.90,1,1,2015,1,30
7,EPWR,16.8858,51.1027,2.0,93.08,8.05,1,1,2015,1,30
8,EPWR,16.8858,51.1027,3.0,86.69,9.20,1,1,2015,2,0
9,EPWA,20.9611,52.1628,1.0,100.0,6.90,1,1,2015,2,0


### Note - columns:

lon - longtitiude

lat - latitude

tmpc - temperature in Celsius

relh - relative humidity

sped - wind speed mph (will be converted to meters per second)

day - day of the month

month - month of the year

year - year

time - hour of the day



### identify and replace Missing values with NaNs

In [3]:
df=filtered.replace('M', np.nan)

nan_count = df.isna().sum()

print(nan_count)

station      0
lon          0
lat          0
tmpc         0
relh       336
sped         0
day          0
month        0
year         0
hour         0
minutes      0
dtype: int64


In [4]:
# 'tmpf' NaN into data from previous timestamp
bool_df = df['relh'].isnull()
indexes = df[bool_df].index
for i in indexes:
    df['relh'][i]=df['relh'][i-1]

nan_count = df.isna().sum()

print(nan_count)

C:\Users\AWeg\AppData\Local\Temp\ipykernel_14852\2032968779.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['relh'][i]=df['relh'][i-1]


station    0
lon        0
lat        0
tmpc       0
relh       0
sped       0
day        0
month      0
year       0
hour       0
minutes    0
dtype: int64


### set types and convert knots to kiloeters per hour

In [5]:
#typesetting + sped conversion

df['lon'] = df['lon'].astype(float)
df['lat'] = df['lat'].astype(float)
df['tmpc'] = df['tmpc'].astype(int)
df['relh'] = df['relh'].astype(float)
df['sped'] = df['sped'].astype(float)
df['sped'] = df['sped'].apply(lambda x: x * 1.852) # knots -> kmph
df['day'] = df['day'].astype(int)
df['month'] = df['month'].astype(int)
df['year'] = df['year'].astype(int)
df['hour'] = df['hour'].astype(int)
df['minutes'] = df['minutes'].astype(int)

df.head()

,station,lon,lat,tmpc,relh,sped,day,month,year,hour,minutes
0,EPWR,16.8858,51.1027,2,86.59,12.7788,1,1,2015,1,0
1,EPRZ,22.0200,50.1114,-4,79.62,23.4278,1,1,2015,1,0
2,EPWA,20.9611,52.1628,1,93.03,12.7788,1,1,2015,1,0
3,EPSC,14.6228,53.3953,4,100.00,17.0384,1,1,2015,1,0
4,EPSC,14.6228,53.3953,4,93.19,17.0384,1,1,2015,1,30


### split data to training and testing, from 2015 to 2021 weather : 2022 for validation

In [6]:
# for learning
df_2015_2021 = df[(df['year'] <= 2021)]
# dates_2015_2021 = dates[df['year'] <= 2021]

# for validation
df_2022 = df[(df['year'] == 2022)]
# dates_2022 = dates[df['year'] == 2022]

df_2015_2021

,station,lon,lat,tmpc,relh,sped,day,month,year,hour,minutes
0,EPWR,16.8858,51.1027,2,86.59,12.7788,1,1,2015,1,0
1,EPRZ,22.0200,50.1114,-4,79.62,23.4278,1,1,2015,1,0
2,EPWA,20.9611,52.1628,1,93.03,12.7788,1,1,2015,1,0
3,EPSC,14.6228,53.3953,4,100.00,17.0384,1,1,2015,1,0
4,EPSC,14.6228,53.3953,4,93.19,17.0384,1,1,2015,1,30
...,...,...,...,...,...,...,...,...,...,...,...
490180,EPWR,16.8858,51.1027,11,93.55,34.0768,31,12,2021,23,0
490181,EPWR,16.8858,51.1027,11,93.55,31.9470,31,12,2021,23,30
490182,EPWA,20.9611,52.1628,10,100.00,27.6874,31,12,2021,23,30
490183,EPRZ,22.0200,50.1114,6,100.00,19.1682,31,12,2021,23,30


### check for imbalannce in number of stations

In [7]:
df_2015_2021.groupby('station')['station'].count()

station
EPRZ    122610
EPSC    122536
EPWA    122535
EPWR    122504
Name: station, dtype: int64

### define the function that for each station generates dataframe shifted by given timedelta(hours), it will be the target variable or the feature depending on is_x value

In [8]:
def generate_dfs(raw: pd.DataFrame, is_x: bool, timedelta):
    dfs=[]
    stations=["EPRZ", "EPSC", "EPWA", "EPWR"]
    #print('start')
    for station in stations:
        wdf=raw[(raw['station']==station) & (raw['minutes']==0)]
        #print(wdf.head())
        if is_x:
            wdf = wdf[:-timedelta]
        else:
            wdf = wdf[timedelta:]
        dfs.append(wdf.copy())
    return pd.concat(dfs).reset_index(drop=True)

# XGBoost model

# Predictions for 12 hours if last_pred_hour = 12
- if last_pred_hour = 3 and uncommented .pickle save => all models are being saved

In [9]:

last_pred_hour = 3


MAE_humid = []
MAE_wind = []
MAE_temp = []

bias_relh = []
bias_sped = []
bias_temp = []

for hour in range(1,last_pred_hour+1):
    x_train_xgb = generate_dfs(df_2015_2021, is_x=True, timedelta=hour)
    y_train_xgb = generate_dfs(df_2015_2021, is_x=False, timedelta=hour)
    x_test_xgb = generate_dfs(df_2022, is_x=True, timedelta=hour)
    y_test_xgb = generate_dfs(df_2022, is_x=False, timedelta=hour)

    y_train_xgb.drop(axis="columns", inplace=True, labels=["station", "minutes"])
    x_train_xgb.drop(axis="columns", inplace=True, labels=["station", "minutes"])
    y_test_xgb.drop(axis="columns", inplace=True, labels=["station", "minutes"])
    x_test_xgb.drop(axis="columns", inplace=True, labels=["station", "minutes"])

    x_train_xgb, y_train_xgb
    reg = xgb.XGBRegressor(
        tree_method="hist",
        n_estimators=200,
        n_jobs=16,
        max_depth=12,
        multi_strategy="multi_output_tree",
        subsample=0.6,
    )
    
    reg.fit(x_train_xgb, y_train_xgb, eval_set=[(x_train_xgb, y_train_xgb)])

    file_name = 'xgb_models/' + "xgb" + str(hour) + ".pkl"
    with open(file_name, "xb") as f_1:
        pickle.dump(reg, f_1, -1)

    y_pred_xgb = reg.predict(x_test_xgb)
    y_pred_xgb = pd.DataFrame(y_pred_xgb, columns=[ "lon", "lat",  "tmpc",  "relh" ,"sped" ,"day","month", "year", "hour"])
    MAE_humid.append(mean_absolute_error(y_pred_xgb[["relh"]],y_test_xgb[["relh"]]))
    MAE_wind.append(mean_absolute_error(y_pred_xgb[["sped"]],y_test_xgb[["sped"]]))
    MAE_temp.append(mean_absolute_error(y_pred_xgb[["tmpc"]],y_test_xgb[["tmpc"]]))


    bias_relh.append(sum((y_pred_xgb[["relh"]].values - y_test_xgb[["relh"]].values)/len(y_test_xgb))[0])
    bias_sped.append(sum((y_pred_xgb[["sped"]].values - y_test_xgb[["sped"]].values)/len(y_test_xgb))[0])
    bias_temp.append(sum((y_pred_xgb[["tmpc"]].values - y_test_xgb[["tmpc"]].values)/len(y_test_xgb))[0])


[10:34:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "multi_strategy" } are not used.

[0]	validation_0-rmse:471.33190
[1]	validation_0-rmse:329.93768
[2]	validation_0-rmse:230.96320
[3]	validation_0-rmse:161.68360
[4]	validation_0-rmse:113.19029
[5]	validation_0-rmse:79.24918
[6]	validation_0-rmse:55.49647
[7]	validation_0-rmse:38.87791
[8]	validation_0-rmse:27.25644
[9]	validation_0-rmse:19.13802
[10]	validation_0-rmse:13.47820
[11]	validation_0-rmse:9.54838
[12]	validation_0-rmse:6.84075
[13]	validation_0-rmse:5.00270
[14]	validation_0-rmse:3.78563
[15]	validation_0-rmse:3.01186
[16]	validation_0-rmse:2.54089
[17]	validation_0-rmse:2.27307
[18]	validation_0-rmse:2.12481
[19]	validation_0-rmse:2.03954
[20]	validation_0-rmse:1.99359
[21]	validation_0-rmse:1.96497
[22]	validation_0-rmse:1.94924
[23]	validation_0-rmse:1.93809
[24]	validation_0-rmse:1.92284
[25]	validation

In [10]:
# relh	 skph 	temp
data = pd.DataFrame()
data['humid'] = MAE_humid
data['wind'] = MAE_wind
data['temp'] = MAE_temp

## Bias only for three  hours here

In [11]:
# relh	 skph 	temp
biases = pd.DataFrame()
biases['humid'] = bias_relh
biases['wind'] = bias_sped
biases['temp'] = bias_temp

# save data to xgb_models folder
biases.to_csv("xgb_models/biases_xgboost")

In [12]:
for i in range(3):
        file_name = 'xgb_models/' + "xgb" + (str)(i+1) + ".pkl"
        print(file_name)
        with open(file_name, "ab") as f_1:
                pickle.dump(biases[biases.index==i], f_1, -1)

xgb_models/xgb1.pkl
xgb_models/xgb2.pkl
xgb_models/xgb3.pkl
